In [1]:
import json
import pandas as pd
import torch
from main import *
import yaml

from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModel
import json
import random
import argparse
from sklearn.metrics import f1_score

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from transformers import LlamaTokenizer, LlamaForSequenceClassification
import torch, sys
from tqdm import tqdm
from transformers import LlamaTokenizer, AutoModel, AutoTokenizer
            
import pickle



/home/phuongnm/SP_LLMs/env_llm/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import  AutoModel, AutoTokenizer,AutoModelForCausalLM
model_name = 'lmsys/vicuna-33b-v1.3'
print("Loading model ...")
tokenizer = AutoTokenizer.from_pretrained(model_name) 
model = AutoModelForCausalLM.from_pretrained(model_name, device_map="auto", load_in_8bit=True)
model.eval()


Loading model ...


You are using the legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This means that tokens that come after special tokens will not be properly handled. We recommend you to read the related pull request available at https://github.com/huggingface/transformers/pull/24565
The model weights are not tied. Please use the `tie_weights` method before using the `infer_auto_device` function.
Loading checkpoint shards: 100%|██████████| 7/7 [02:11<00:00, 18.77s/it]


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 6656, padding_idx=0)
    (layers): ModuleList(
      (0-59): 60 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=6656, out_features=6656, bias=False)
          (k_proj): Linear8bitLt(in_features=6656, out_features=6656, bias=False)
          (v_proj): Linear8bitLt(in_features=6656, out_features=6656, bias=False)
          (o_proj): Linear8bitLt(in_features=6656, out_features=6656, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=6656, out_features=17920, bias=False)
          (up_proj): Linear8bitLt(in_features=6656, out_features=17920, bias=False)
          (down_proj): Linear8bitLt(in_features=17920, out_features=6656, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSN

In [4]:

class BatchPreprocessorLLM(BatchPreprocessor): 
    def __init__(self, tokenizer, dataset_name=None, window_ct=2, emotion_labels=[]) -> None:
        self.tokenizer = tokenizer
        self.separate_token_id = self.tokenizer.convert_tokens_to_ids("</s>")
        self.dataset_name  = dataset_name
        self.window_ct = window_ct
        self.emotion_labels = emotion_labels
        self.printted = False
    
    @staticmethod
    def load_raw_data(path_data):
        raw_data = json.load(open(path_data))
        if isinstance(raw_data, dict):
            new_data_list = []
            for k, v in raw_data.items():
                v['s_id'] = k
                new_data_list.append(v)
            return new_data_list
        elif isinstance(raw_data, list):
            return raw_data
    
    @staticmethod
    def get_speaker_name(s_id, gender, data_name):
        if data_name == "iemocap":
            # iemocap: label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
            speaker = {
                        "Ses01": {"F": "Mary", "M": "James"},
                        "Ses02": {"F": "Patricia", "M": "John"},
                        "Ses03": {"F": "Jennifer", "M": "Robert"},
                        "Ses04": {"F": "Linda", "M": "Michael"},
                        "Ses05": {"F": "Elizabeth", "M": "William"},
                    }
            s_id_first_part = s_id[:5]
            return speaker[s_id_first_part][gender].upper()
        elif data_name in ['meld', "emorynlp"]:
            # emorynlp: label index mapping =  {'Joyful': 0, 'Mad': 1, 'Peaceful': 2, 'Neutral': 3, 'Sad': 4, 'Powerful': 5, 'Scared': 6}
            # meld: label index mapping = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
            gender_idx = gender.index(1) 
            return f"SPEAKER_{gender_idx}"
        elif data_name=='dailydialog':
            # dailydialog:  {'no_emotion': 0, 'happiness': 1, 'sadness': 2, 'surprise': 3,  'anger': 4, 'fear': 5, 'disgust':6}
            return f"SPEAKER_{gender}"
        
    def sentence_mixed_by_surrounding(self, sentences, around_window, s_id, genders, data_name):
        new_conversations = []
        align_sents = []
        for i, cur_sent in enumerate(sentences):
            tmp_s = ""
            for j in range(max(0, i-around_window), min(len(sentences), i+around_window+1)):
                u_j =  f"{self.get_speaker_name(s_id, genders[j], data_name=data_name)}: {sentences[j]}"
                if i == j:
                    align_sents.append(u_j)
                tmp_s +=  f"\n{u_j}"
            new_conversations.append(tmp_s)
        return new_conversations, align_sents
    
    def __call__(self, batch):
        raw_sentences = []
        raw_sentences_flatten = []
        labels = []
        speaker_info = []
        listener_info = []

        # masked tensor  
        lengths = [len(sample['sentences']) for sample in batch]
        max_len_conversation = max(lengths)
        padding_utterance_masked = torch.BoolTensor([[False]*l_i+ [True]*(max_len_conversation - l_i) for l_i in lengths])

        # collect all sentences
        # - intra speaker
        flatten_data = []
        intra_speaker_masekd_all = torch.BoolTensor(len(batch), max_len_conversation,max_len_conversation)
        for i, sample in enumerate(batch):
            new_conversations, align_sents = self.sentence_mixed_by_surrounding(sample['sentences'], 
                                                                        around_window=self.window_ct, 
                                                                        s_id=sample['s_id'], 
                                                                        genders=sample['genders'],
                                                                        data_name=self.dataset_name)
            few_shot_example = """\n=======
Context: Given predefined emotional label set [happy, sad, neutral, angry, excited, frustrated], and bellow conversation: 
"
PATRICIA: You know, it's lovely here, the air is sweet.
PATRICIA: No, not sorry.  But, um. But I'm not gonna stay.
JOHN: The trouble is, I planned on sort of sneaking up on you on a period of a week or so.  But they take it for granted that we're all set.
PATRICIA: I knew they would, your mother anyway.
PATRICIA: Well, from her point of view, why else would I come?
PATRICIA: I guess this is why I came.
JOHN: I'm embarrassing you and I didn't want to tell it to you here.  I wanted some place we'd never been before.  A place where we'd be brand new to each other.
PATRICIA: Well, you started to write me
JOHN: You felt something that far back?
PATRICIA: Every day since.
JOHN: Ann, why didn't you let me know?
JOHN: Let's drive someplace.  I want to be alone with you.
JOHN: No.  Nothing like that.
"

Question: What is the emotion of the speaker at the utterance "PATRICIA: Well, from her point of view, why else would I come?"?
Answer: neutral

Question: What is the emotion of the speaker at the utterance "PATRICIA: I guess this is why I came."?
Answer: happy

Question: What is the emotion of the speaker at the utterance "JOHN: I'm embarrassing you and I didn't want to tell it to you here.  I wanted some place we'd never been before.  A place where we'd be brand new to each other."?
Answer: excited
"""
            for i_u, (conv, utterance) in enumerate(zip(new_conversations, align_sents)):
                prompt_extract_context_vect = few_shot_example + f"\n=======\nContext: Given predefined emotional label set [{', '.join(self.emotion_labels)}], and bellow conversation:\n\"{conv}\n\"\n\nQuestion: What is the emotion of the speaker at the utterance \"{utterance}\"?\nAnswer:" 
                if not self.printted:
                    print(prompt_extract_context_vect)
                    self.printted = True 
                    
                inputs = self.tokenizer(prompt_extract_context_vect, return_tensors="pt")
                input_ids = inputs["input_ids"] 
                flatten_data.append({
                    "s_id": sample['s_id'],
                    "u_idx": i_u,
                    "prompt_content": prompt_extract_context_vect,
                    "input_ids": input_ids,
                    }
                )
                
        return flatten_data

In [3]:

class BatchPreprocessorLLMSpeakerDescription(BatchPreprocessor): 
    def __init__(self, tokenizer, dataset_name=None, window_ct=2, emotion_labels=[]) -> None:
        self.tokenizer = tokenizer
        self.separate_token_id = self.tokenizer.convert_tokens_to_ids("</s>")
        self.dataset_name  = dataset_name
        self.window_ct = window_ct
        self.emotion_labels = emotion_labels
    
    @staticmethod
    def load_raw_data(path_data):
        raw_data = json.load(open(path_data))
        if isinstance(raw_data, dict):
            new_data_list = []
            for k, v in raw_data.items():
                v['s_id'] = k
                new_data_list.append(v)
            return new_data_list
        elif isinstance(raw_data, list):
            return raw_data
    
    @staticmethod
    def get_speaker_name(s_id, gender, data_name):
        if data_name == "iemocap":
            # iemocap: label index mapping = {'hap':0, 'sad':1, 'neu':2, 'ang':3, 'exc':4, 'fru':5}
            speaker = {
                        "Ses01": {"F": "Mary", "M": "James"},
                        "Ses02": {"F": "Patricia", "M": "John"},
                        "Ses03": {"F": "Jennifer", "M": "Robert"},
                        "Ses04": {"F": "Linda", "M": "Michael"},
                        "Ses05": {"F": "Elizabeth", "M": "William"},
                    }
            s_id_first_part = s_id[:5]
            return speaker[s_id_first_part][gender].upper()
        elif data_name in ['meld', "emorynlp"]:
            # emorynlp: label index mapping =  {'Joyful': 0, 'Mad': 1, 'Peaceful': 2, 'Neutral': 3, 'Sad': 4, 'Powerful': 5, 'Scared': 6}
            # meld: label index mapping = {'neutral': 0, 'surprise': 1, 'fear': 2, 'sadness': 3, 'joy': 4, 'disgust': 5, 'anger':6}
            gender_idx = gender.index(1) 
            return f"SPEAKER_{gender_idx}"
        elif data_name=='dailydialog':
            # dailydialog:  {'no_emotion': 0, 'happiness': 1, 'sadness': 2, 'surprise': 3,  'anger': 4, 'fear': 5, 'disgust':6}
            return f"SPEAKER_{gender}"
        
    def preprocess(self, all_conversations):
        
        new_data = {}
        gr_by_len = {}
        for i, sample in enumerate(all_conversations):

            all_utterances = []
            all_speaker_names= []
            for i_u, u in enumerate(sample['sentences']):
                speaker_name = self.get_speaker_name(sample['s_id'], sample['genders'][i_u], self.dataset_name)
                u_full_name = f'{speaker_name}: {u}'
                all_utterances.append(u_full_name)
                all_speaker_names.append(speaker_name) 
            
            full_conversation = "\n".join(all_utterances)
            prompts_speaker_description_word_ids = {}
            prompting_input = {}
            for speaker_name in set(all_speaker_names):
                prompting ="Given this conversation between speakers: \n\"" + full_conversation + "\n\"\nIn overall of above conversation, what do you think about the characteristics speaker {}? (Note: provide an answer within 250 words)".format(speaker_name) 
                prompts_speaker_description_word_ids[speaker_name] = self.tokenizer(prompting, return_tensors="pt")["input_ids"]
                prompting_input[speaker_name] = prompting
                
                # group by len for batch decode by llm 
                if prompts_speaker_description_word_ids[speaker_name].shape[-1] not in  gr_by_len:
                    gr_by_len[prompts_speaker_description_word_ids[speaker_name].shape[-1]] = []
                gr_by_len[prompts_speaker_description_word_ids[speaker_name].shape[-1]].append({
                    'w_ids': prompts_speaker_description_word_ids[speaker_name],
                    'conv_id': sample['s_id'],
                    'type_data': sample['type_data'],
                    "prompting_input": prompting,
                    'speaker_name': speaker_name,
                    'all_speaker_names': all_speaker_names
                })
                
        return gr_by_len
    

In [ ]:
import json 
import os 
import traceback

dataset_name = 'meld'
data_folder = '../data/'

raw_data = []
for type_data in ['valid', 'test', 'train']:
    data_name_pattern= f'{dataset_name}.{type_data}'
    path_processed_data = f'{data_folder}/llm_vectors/{data_name_pattern}_spdesc_{model_name.split("/")[-1]}.json'
    
    org_raw_data = BatchPreprocessorLLMSpeakerDescription.load_raw_data(f"{data_folder}/{data_name_pattern}.json")
    
    if os.path.exists(path_processed_data):
        processed_data = json.load(open(path_processed_data, 'rt'))
        print(f'- sucessful processed {len(processed_data)}/{len(org_raw_data)} conversations in data-type ={type_data}')
        json.dump(processed_data, open(path_processed_data+"_backup.json", 'wt'), indent=2)
        org_raw_data = [e for e in org_raw_data if e['s_id'] not in processed_data]
        
    print(f'- Continue process {len(org_raw_data)} conversations in data-type ={type_data}')
    for e in org_raw_data:
        e['type_data'] = type_data
    raw_data = raw_data + org_raw_data
    
data_preprocessor = BatchPreprocessorLLMSpeakerDescription(tokenizer, dataset_name=dataset_name, window_ct=4, 
                                            emotion_labels=['happy', 'sad', 'neutral', 'angry', 'excited', 'frustrated'])

gr_by_len  = data_preprocessor.preprocess(raw_data)
all_data = {}
for len_promting, speaker_promts in tqdm(gr_by_len.items()):
    for batch_size in [8, 5, 2, 1]:
        try:
            all_promtings_texts = [e['prompting_input'] for e in speaker_promts]
            data_loader = DataLoader(all_promtings_texts, 
                                batch_size=batch_size, 
                                shuffle=False)
            output_sp_desc = []
            with torch.no_grad():
                for i, speaker_promts_in_batch in enumerate(data_loader):
                    # batch decoded by llm 
                    inputs = tokenizer(speaker_promts_in_batch, return_tensors="pt", padding=False)
                    input_ids = inputs["input_ids"].to("cuda")
                    with torch.no_grad():
                        outputs = model.generate(input_ids, max_new_tokens=200, temperature=1.0)
                    output_text = tokenizer.batch_decode(outputs)
                    
                    for j, e in enumerate(output_text):
                        output_sp_desc.append(e.replace(all_promtings_texts[j], ""))
            
                for i, out in enumerate(output_sp_desc):
                    speaker_promts[i]['sp_desc'] = out  
            break
                    
        except Exception as e:
            traceback.print_exc()
            print(e)
            if batch_size == 1:
                print(["Errr "]*10)
                
for type_data in ['valid', 'test', 'train']:
    data_name_pattern= f'{dataset_name}.{type_data}'
    path_processed_data = f'{data_folder}/llm_vectors/{data_name_pattern}_spdesc_{model_name.split("/")[-1]}.json'
    
    processed_data = {}
    if os.path.exists(path_processed_data):
        processed_data = json.load(open(path_processed_data, 'rt'))
        print(f'- load processed [old] {len(processed_data)} conversations in data-type ={type_data}')
        
    all_data = {}
    for len_promting, speaker_promts in gr_by_len.items():
        for description in speaker_promts:
            if type_data != description['type_data']:
                continue
            
            if description['conv_id'] not in all_data:
                all_data[description['conv_id']] = {
                    'all_speaker_names': description['all_speaker_names'],
                    'vocab_sp2desc':  {}
                }
            all_data[description['conv_id']]['vocab_sp2desc'][description['speaker_name']] = description['sp_desc']
    
    print(f'- sucessful processed [new] {len(all_data)} conversations in data-type ={type_data}')
    # json.dump(all_data, open(f'{path_data}_new.json', 'wt'), indent=2)

    all_data_new = {}
    for k, v in all_data.items():
        all_data_new[k] = []
        for sp_name in v['all_speaker_names']:
            all_data_new[k].append(v['vocab_sp2desc'][sp_name])
            
    print(f'- update processed [new] {len(all_data_new)} + [old] {len(processed_data)} conversations in data-type ={type_data}')
    all_data_new.update(processed_data)
    json.dump(all_data_new, open(f'{path_processed_data}', 'wt'), indent=2)
        

- Continue process 114 conversations in data-type =valid
- Continue process 280 conversations in data-type =test
- Continue process 1038 conversations in data-type =train


  0%|          | 0/441 [00:00<?, ?it/s]

In [13]:
# import json 
# import os 

# dataset_name = 'meld'
# data_folder = '../data/all_raw_data/'


# for type_data in ['valid','train','test']:
#     data_name_pattern= f'{dataset_name}.{type_data}'
    
    
#     raw_data = BatchPreprocessor.load_raw_data(f"{data_folder}/{data_name_pattern}.json")
    
#     data_preprocessor = BatchPreprocessorLLMSpeakerDescription(tokenizer, dataset_name=dataset_name, window_ct=4, 
#                                              emotion_labels=['happy', 'sad', 'neutral', 'angry', 'excited', 'frustrated'])
#     all_data = {}
#     test_loader = DataLoader(raw_data, 
#                              batch_size=1, 
#                              collate_fn=data_preprocessor, 
#                              shuffle=False)
    
#     with torch.no_grad():
#         path_data = f'{data_folder}/llm_vectors/{data_name_pattern}_speaker_descriptions.json'
#         for i, convs in enumerate(tqdm(test_loader)):
#             if os.path.exists(path_data):
#                 all_data = json.load(open(path_data, 'rt'))
#             for j, conv in enumerate(convs):
#                 if conv['s_id'] in all_data:
#                     continue
#                 speaker_descriptions = []
#                 map_speaker2descriptions = {}

#                 for speaker_name, prompt_speaker_des_ids in conv['prompts_speaker_description_word_ids'].items():
#                     input_ids = prompt_speaker_des_ids.to("cuda") 
#                     outputs = model.generate(input_ids, max_new_tokens=300)
#                     map_speaker2descriptions[speaker_name] = tokenizer.decode(outputs[0])

#                 for sp_name in conv['speaker_names']:
#                     speaker_descriptions.append(map_speaker2descriptions[sp_name].replace(conv['prompting_input'][sp_name], ''))
            
#                 all_data[conv['s_id']] = speaker_descriptions
#             json.dump(all_data, open(f'{data_folder}/llm_vectors/{data_name_pattern}_speaker_descriptions.json', 'wt'))

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /JOBs/tmpdir/pbs.7884080.spcc-adm1/ipykernel_3824609/16680716.py:12 in <cell line: 8>            │
│                                                                                                  │
│ [Errno 2] No such file or directory:                                                             │
│ '/JOBs/tmpdir/pbs.7884080.spcc-adm1/ipykernel_3824609/16680716.py'                               │
│                                                                                                  │
│ /home/phuongnm/per_erc/src/main.py:78 in load_raw_data                                           │
│                                                                                                  │
│    75 │                                                                                          │
│    76 │   @staticmethod                                                                          │
│    77 │   def load_raw_data(path_data):                                                          │
│ ❱  78 │   │   raw_data = json.load(open(path_data))                                              │
│    79 │   │   if isinstance(raw_data, dict):                                                     │
│    80 │   │   │   new_data_list = []                                                             │
│    81 │   │   │   for k, v in raw_data.items():                                                  │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
FileNotFoundError: [Errno 2] No such file or directory: '../data/all_raw_data//meld.valid.json'

In [ ]:


# dataset_name = 'iemocap'
# data_folder = '/home/phuongnm/per_erc/data/'


# for type_data in [ 'train', 'valid', 'test']:
#     data_name_pattern= f'{dataset_name}.{type_data}'
    
    
#     raw_data = BatchPreprocessor.load_raw_data(f"{data_folder}/{data_name_pattern}.json")
    
#     data_preprocessor = BatchPreprocessorLLM(tokenizer, dataset_name=dataset_name, window_ct=4, 
#                                              emotion_labels=['happy', 'sad', 'neutral', 'angry', 'excited', 'frustrated'])
#     all_data = {}
    
#     test_loader = DataLoader(raw_data, 
#                              batch_size=1, 
#                              collate_fn=data_preprocessor, 
#                              shuffle=False)
#     with torch.no_grad():
#         for i, conv in enumerate(tqdm(test_loader)):
#             llm_vectors = []
#             for i_s, sent in enumerate(conv):
#                 input_ids = sent['input_ids'].to("cuda")
#                 outputs = model(input_ids)
#                 sent.pop('input_ids')
#                 llm_vectors.append(outputs.last_hidden_state[0][0].cpu())
        
#             all_data[conv[0]['s_id']] = llm_vectors
#     pickle.dump(all_data, open(f'{data_folder}/llm_vectors/{data_name_pattern}_llm_fewshot.pkl', 'wb'))


In [ ]:

    #  q_input_group_by_len = {}
    #     for i_check, support_samples in enumerate(tqdm(all_candidate_examples)):
    #         utterances.append(support_samples['q_input'])
    #         if i_check not in query_ids:
    #             continue
    #         str_query, max_new_tokens, num_demonstrations, q_sub_tok_len = self.query_generate_with_support_samples(support_samples, 
    #                                                                                                                 args.top_k_demos, 
    #                                                                                                                 50, 
    #                                                                                                                 missing_predicate[i_check] if missing_predicate is not None else None)
    #         if q_sub_tok_len not in q_input_group_by_len:
    #             q_input_group_by_len[q_sub_tok_len] = []
                
    #         q_input_group_by_len[q_sub_tok_len].append([str_query, max_new_tokens, num_demonstrations, q_sub_tok_len, i_check])
                    
    #     for q_len, gr_info in tqdm(q_input_group_by_len.items()):
    #         q_outputs = []
            
    #         try:
    #             # try to infer with llm
    #             for batch_info in DataLoader(gr_info, batch_size=args.batch_size, shuffle=False):
    #                 batch_queries = batch_info[0]
    #                 batch_max_new_tokens = max(batch_info[1])
    #                 batch_output =  self.batch_query(batch_queries, batch_max_new_tokens)    # TODO CHECK
    #                 q_outputs += batch_output
    #         except Exception as e:
    #             traceback.print_exc()
    #             print(e)
    #             continue  # if exception by out of CUDA MEMORY 